In [2]:
import cv2
import numpy as np
import os
import glob
import json
import shutil
from pathlib import Path
from tqdm import tqdm

def find_project_root(marker=".gitignore"):
    current = Path.cwd()
    for parent in [current] + list(current.parents):
        if (parent / marker).exists():
            return parent.resolve()
    raise FileNotFoundError(
        f"Project root marker '{marker}' not found starting from {current}")
    

In [3]:
root = find_project_root()

intrinsic_path = os.path.join(root, "output", "intrinsic_params.json")
with open(intrinsic_path, "r") as f:
    intrinsics = json.load(f)

extrinsic_path = os.path.join(root, "output", "keep_best_stereo_params.json")
with open(extrinsic_path, "r") as f:
    stereo_extrinsics = json.load(f)

In [8]:

def compute_projection_matrix(K, R, t):
    # Ensure t has shape (3,1)
    t = t.reshape(3, 1)
    # Stack R (3x3) and t (3x1) to form a 3x4 matrix
    RT = np.hstack((R, t))
    P = K @ RT
    return P.astype(np.float32)

def triangulate_ball_position(P_left, P_right, pt_left, pt_right):
    # Convert points to homogeneous coordinates (2x1 vectors)
    pt_left_hom = np.array([[pt_left[0]], [pt_left[1]], [1.0]])
    pt_right_hom = np.array([[pt_right[0]], [pt_right[1]], [1.0]])
    
    # Triangulate using OpenCV
    pts_4d_hom = cv2.triangulatePoints(P_left, P_right, pt_left_hom[:2], pt_right_hom[:2])
    # Convert from homogeneous coordinates to 3D
    pts_3d = pts_4d_hom / pts_4d_hom[3]
    return pts_3d[:3].flatten()


K_left1 = np.array(intrinsics["LEFT_CAM_A"]["K"])
K_right1 = np.array(intrinsics["RIGHT_CAM_A"]["K"])

R_left1 = np.array(stereo_extrinsics["stereo_left"]["rotation_matrix"])
t_left1 = np.array(stereo_extrinsics["stereo_left"]["translation_vector"])

R_right1 = np.array(stereo_extrinsics["stereo_right"]["rotation_matrix"])
t_right1 = np.array(stereo_extrinsics["stereo_right"]["translation_vector"])

P_left_pair1 = compute_projection_matrix(K_left1, R_left1, t_left1)
P_right_pair1 = compute_projection_matrix(K_right1, R_right1, t_right1)

print("P_left shape:", P_left_pair1.shape)
print("P_right shape:", P_right_pair1.shape)


pt_left_pair1 = (100, 150) 
pt_right_pair1 = (110, 148)

ball_pos_pair1 = triangulate_ball_position(P_left_pair1, P_right_pair1, pt_left_pair1, pt_right_pair1)
print("3D Ball Position from Stereo Pair 1:", ball_pos_pair1)

P_left shape: (3, 4)
P_right shape: (3, 4)
3D Ball Position from Stereo Pair 1: [514.42968782 169.43526952 250.46766262]


In [7]:
def reproject_3d_point(P, point3d):
    # Convert to homogeneous coordinates: (X, Y, Z) -> (X, Y, Z, 1)
    point3d_hom = np.append(point3d, 1)
    # Project: (3x4) * (4x1) = (3x1)
    point2d_hom = P @ point3d_hom
    # Normalize: (x, y, w) -> (x/w, y/w)
    point2d = point2d_hom[:2] / point2d_hom[2]
    return point2d

# Reproject using the left and right projection matrices:
reproj_left = reproject_3d_point(P_left_pair1, ball_pos_pair1)
reproj_right = reproject_3d_point(P_right_pair1, ball_pos_pair1)

print("Reprojected point in left image:", reproj_left)
print("Reprojected point in right image:", reproj_right)

Reprojected point in left image: [ 73.43574523 559.82038481]
Reprojected point in right image: [-1060.03875926  4567.83726533]
